# Creación de su primer recomendador de videos bajo demanda

Este cuaderno lo guiará a través de los pasos para crear un grupo de conjuntos de datos de dominio y un
recomendador que devuelva recomendaciones de películas en función de los datos recopilados del conjunto de datos de MovieLens. El objetivo es recomendar películas relevantes para un usuario concreto.

Los datos provienen del [proyecto de MovieLens](https://grouplens.org/datasets/movielens/). Para obtener más información sobre los datos y sus posibles usos, siga el enlace.

# Cómo utilizar el cuaderno

El código se divide en celdas como la siguiente. En la parte superior de esta página, puede hacer clic en un botón triangular para Ejecutar cada celda y pasar a la siguiente, o puede presionar `Shift` + `Enter` mientras está en la celda para ejecutarla y pasar a la siguiente.

A medida que una celda se ejecuta, verá una línea al costado que muestra un `*` mientras la celda se está ejecutando o se actualizará a un número para indicar la última celda que terminó de ejecutarse luego de que haya terminado de ejecutar todo el código dentro de una celda.

Solo debe seguir las instrucciones que aparecen a continuación y ejecutar las celdas para comenzar a utilizar Amazon Personalize con recomendadores optimizados para el caso.

## Importaciones
Python viene con una amplia colección de bibliotecas y necesitamos importarlas, así como las que se instalan para ayudarnos, como [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK para Python) y [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), que son herramientas básicas de la ciencia de datos.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

A continuación, querrá validar que su entorno puede comunicarse correctamente con Amazon Personalize. Las líneas que figuran a continuación hacen precisamente eso.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configurar los datos
Los datos se importan a Amazon Personalize a través de Amazon S3. A continuación, especificaremos un bucket creado dentro de AWS para este ejercicio.
A continuación, se actualizará la variable `bucket` para que en su lugar se establezca el valor que creó antes en los pasos de CloudFormation. Esto debe estar en un archivo de texto de su trabajo anterior. No es necesario cambiar el `filename`.
### Especificar un bucket y una ubicación de salida de datos
Actualice el nombre del `bucket` por un nombre exclusivo.

In [ ]:
filename = "movie-lens-100k.csv"

## Descargar, preparar y cargar los datos de entrenamiento
En la actualidad, aún no tiene los datos de MovieLens cargados de forma local para examinarlos. Ejecute las siguientes líneas para descargar la última copia y examinarla rápidamente.

### Descargar y explorar el conjunto de datos

In [ ]:
!wget -N https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

In [ ]:
!ls ml-latest-small

In [ ]:
!pygmentize ml-latest-small/README.txt

In [ ]:
interactions_data = pd.read_csv('./ml-latest-small/ratings.csv')
pd.set_option('display.max_rows', 5)
interactions_data

In [ ]:
interactions_data.info()

## Preparar los datos

### Datos de las interacciones
Como puede ver, los datos contienen un UserID, un ItemID, una clasificación y una marca temporal.

Ahora eliminaremos los elementos con clasificaciones bajas y también la columna Rating (Clasificación) antes de crear nuestro modelo.

También agregaremos la columna EVENT_TYPE a todas las interacciones.

In [ ]:
interactions_data = interactions_data[interactions_data['rating'] > 3]                # Keep only movies rated higher than 3 out of 5.
interactions_data = interactions_data[['userId', 'movieId', 'timestamp']]
interactions_data.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True)
interactions_data['EVENT_TYPE']='watch' #Adding an EVENT_TYPE column that has the event type "watched" for all movies
interactions_data.head()

### Metadatos del elemento

Abra el archivo de datos del elemento y observe las primeras filas.

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

In [ ]:
items_data.info()

In [ ]:
items_data['year'] = items_data['title'].str.extract('.*\((.*)\).*',expand = False)
items_data.head(5)

Seleccione una fecha actual como marca de tiempo de creación para este ejemplo, ya que se desconoce la marca temporal de creación real. En su caso de uso, proporcione la marca temporal de creación adecuada.

In [ ]:
ts= datetime.datetime(2022, 1, 1, 0, 0).strftime('%s')
print(ts)

In [ ]:
items_data["CREATION_TIMESTAMP"] = ts
items_data

In [ ]:
# removing the title
items_data.drop(columns="title", inplace = True)

# renaming the columns to match schema
items_data.rename(columns = { 'movieId':'ITEM_ID', 'genres':'GENRES',
                              'year':'YEAR'}, inplace = True)
items_data

# Metadatos del usuario

El conjunto de datos no tiene ningún metadato de usuario, por lo que crearemos un campo de metadatos falso.

In [ ]:
# get user ids from the interaction dataset

user_ids = interactions_data['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]=user_ids
user_data

## Incorporación de metadatos
El conjunto de datos actual no contiene información adicional del usuario. Para este ejemplo, asignaremos un género a los usuarios de forma aleatoria con la misma probabilidad de ser hombre o mujer.

In [ ]:
possible_genders = ['female', 'male']
random = np.random.choice(possible_genders, len(user_data.index), p=[0.5, 0.5])
user_data["GENDER"] = random
user_data

## Configurar un bucket de S3 y un rol de IAM

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora. Tan solo defina la región como una cadena a continuación.

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled this should be the same region as the Amazon Personalize resources
# region = "us-east-1"


In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedvod"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except:
    print("Bucket already exists. Using bucket", bucket_name)

### Cargar los datos a S3
Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos.

In [ ]:
interactions_filename = "interactions.csv"
interactions_data.to_csv(interactions_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

items_filename = "items.csv"
items_data.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

user_filename = "users.csv"
user_data.to_csv(user_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_filename).upload_file(user_filename)

## Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

Nota: Asegúrese de que el rol que está utilizando para ejecutar el código en este cuaderno tiene los permisos necesarios para modificar la política del bucket de S3.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## Crear y esperar el grupo de conjuntos de datos
La agrupación más grande en Personalize es un grupo de conjuntos de datos, esto aislará sus datos, rastreadores de eventos, soluciones, recomendadores y campañas. Agrupe elementos que comparten una colección de datos común. Siéntase libre de modificar el nombre que aparece a continuación si lo desea.

### Crear grupo de conjuntos de datos

In [ ]:
response = personalize.create_dataset_group(
    name='personalize-video-on-demand-ds-group',
    domain='VIDEO_ON_DEMAND'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Esperar a que el grupo de conjuntos de datos tenga un estado ACTIVO
Debe estar activo antes de que podamos utilizar el grupo de conjunto de datos en cualquiera de los siguientes artículos. Ejecute la celda a continuación y espere a que se muestre activo.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Crear un esquema de interacciones
Un componente central de cómo Personalize entiende sus datos proviene del esquema que se define a continuación. Esta configuración indica al servicio cómo digerir los datos proporcionados a través de su archivo CSV. Observe que las columnas y los tipos se alinean con lo que había en el archivo que creó anteriormente.

In [ ]:
schema = {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "EVENT_TYPE",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_interactions_schema_response = personalize.create_schema(
    name='personalize-demo-interactions-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(json.dumps(create_interactions_schema_response, indent=2))

# Crear un esquema para los elementos (películas)

In [ ]:
schema = {
  "type": "record",
  "name": "Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
    {
      "name": "ITEM_ID",
      "type": "string"
    },
    {
      "name": "GENRES",
      "type": [
        "string"
      ],
      "categorical": True
    },
    {
      "name": "YEAR",
      "type": [
        "string"
      ],
      "categorical": True
    }, 
    {
      "name": "CREATION_TIMESTAMP",
      "type": "long"
    }
  ],
  "version": "1.0"
}
create_items_schema_response = personalize.create_schema(
    name='personalize-demo-items-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

items_schema_arn = create_items_schema_response['schemaArn']
print(json.dumps(create_items_schema_response, indent=2))

# Crear un esquema para los usuarios

In [ ]:
schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
    ],
    "version": "1.0"
}
create_users_schema_response = personalize.create_schema(
    name='personalize-demo-users-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

users_schema_arn = create_users_schema_response['schemaArn']
print(json.dumps(create_users_schema_response, indent=2))

## Crear conjuntos de datos
Después del grupo, lo siguiente que hay que crear son los conjuntos de datos propiamente dichos.

### Crear un conjunto de datos de interacciones

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Crear un conjunto de datos para los elementos

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Crear un conjunto de datos para los usuarios

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Crear un rol de Personalize
Además, Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Las líneas que aparecen a continuación otorgan eso.

Nota: Asegúrese de que el rol que está utilizando para ejecutar el código en este cuaderno tiene los permisos necesarios para crear un rol.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleVODDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Importar los datos
Anteriormente, creó el grupo de conjunto de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos de S3 en Amazon Personalize para su uso, a medida que crea su modelo.
### Crear un trabajo de importación de conjuntos de datos de interacciones

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Crear un trabajo de importación de conjuntos de datos de elementos

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### Crear un trabajo de importación de conjuntos de datos de los usuarios

In [ ]:
create_users_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-users",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_filename)
    },
    roleArn = role_arn
)

dataset_users_import_job_arn = create_users_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_users_dataset_import_job_response, indent=2))

Esperar a que el trabajo de importación de conjuntos de datos tenga un estado ACTIVO
Puede pasar un tiempo antes de que el trabajo de importación se complete. Espere hasta que vea que está activo a continuación.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Interactions DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Items DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_users_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Users DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Elegir un caso de uso del recomendador

Cada dominio tiene diferentes casos de uso. Cuando se crea un recomendador, se crea para un caso de uso específico, y cada caso de uso tiene diferentes requisitos para obtener recomendaciones.


In [ ]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND') # See a list of recommenders for the domain. 
if (len(available_recipes["recipes"])==0):
    # This is a workaround to get the recipes in case 'available_recipes["recipes"]'does not retrieve them
    available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND', nextToken=available_recipes["nextToken"])
display(available_recipes["recipes"])
    

Crearemos un recomendador del tipo "Más como X". Este tipo de recomendador ofrece recomendaciones de videos que son similares a un video que vio un usuario. Con este caso de uso, Amazon Personalize filtra de forma automática los videos que el usuario vio en función del userId especificado en la llamada `get_recommendations`. Para obtener un mejor rendimiento, registre los eventos Clics además de los eventos Vistas requeridos.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'more_like_x_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  datasetGroupArn = dataset_group_arn
)
recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Crearemos un segundo recomendador del tipo "Principales selecciones para usted". Este tipo de recomendador ofrece recomendaciones personalizadas de contenido de streaming para un usuario que especifique. Con este caso de uso, Amazon Personalize filtra automáticamente los videos que el usuario vio en función del userId que especifique y los eventos `Watch`.

[Más casos de uso por dominio](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'top_picks_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
  datasetGroupArn = dataset_group_arn
)
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Esperamos a que los recomendadores terminen de crear y tengan estado `ACTIVE`. Comprobamos periódicamente el estado del recomendador.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_more_like_x_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_more_like_x_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_more_like_x_arn))

    if status == "ACTIVE":
        break
    else:
        print("The More Like X Recommender build is still in progress")
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_top_picks_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_top_picks_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_top_picks_arn))

    if status == "ACTIVE":
        break
    else:
        print("The Top Pics for You Recommender build is still in progress")
        
    time.sleep(60)

# Obtención de recomendaciones con un recomendador
Ahora que los recomendadores se han formado, ¡veamos las recomendaciones que podemos obtener para nuestros usuarios!

In [ ]:
# reading the original data in order to have a dataframe that has both movie_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./ml-latest-small/movies.csv')
items_df.sample(10)

In [ ]:
def get_movie_by_id(movie_id, movie_df):
    """
    This takes in an movie_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[movie_df["movieId"]==int(movie_id)]['title'].values[0]
    except:
        print (movie_id)
        return "Error obtaining title"

### Obtengamos algunas recomendaciones “Más como X”:

In [ ]:
# First pick a user
test_user_id = "1"

# Select a random item
test_item_id = "81847" #Iron Man 59315, Tangled: 81847

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_more_like_x_arn,
    userId = test_user_id,
    itemId = test_item_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_movie_by_id(test_item_id, items_df)])

pd.options.display.max_rows = 20
display(user_recommendations_df)

### Obtenga recomendaciones del recomendador que devuelve las "Principales selecciones para usted":

Si agrega los metadatos del usuario a nuestro usuario de muestra, puede utilizar este tipo de metadatos para obtener información sobre sus usuarios.

In [ ]:
users_data_df = pd.read_csv('./users.csv')

def get_gender_by_id(user_id, user_df):
    """
    This takes in a user_id and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    try:
        return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    except:
        print (user_id)
        return "Error obtaining title"

In [ ]:
# First pick a user
test_user_id = "111" # samples users: 55, 75, 76, 111

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_top_picks_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

column_name = test_user_id+" ("+get_gender_by_id(test_user_id, users_data_df)+")"

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [column_name])

pd.options.display.max_rows =20
display(user_recommendations_df)

## Revisión
Con los códigos anteriores, logró entrenar correctamente un modelo de aprendizaje profundo para generar recomendaciones de elementos basadas en el comportamiento previo del usuario. Ha creado dos recomendadores para dos casos de uso fundamentales. 
En adelante, puede adaptar este código para crear otros recomendadores.

## Notas para el próximo cuaderno:
Hay algunos valores que necesitará para el siguiente cuaderno, ejecute la celda a continuación para almacenarlos y así poder utilizarlos en el cuaderno `Clean_Up_Resources.ipynb`.

Esto sobreescribirá cualquier dato almacenado para esas variables y las ajustará a los valores especificados en este cuaderno. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

Si ha ejecutado el cuaderno `Building_Your_First_Recommender_Ecommerce.ipynb`, asegúrese de volver a ejecutar el paso anterior en el cuaderno `Building_Your_First_Recommender_Ecommerce.ipynb` y volver a ejecutar `Clean_Up_Resources.ipynb` para eliminar los recursos creados en ese cuaderno luego de ejecutar `Clean_Up_Resources.ipynb` con los recursos creados aquí.